In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from sympy import *
from statsmodels.stats.proportion import proportions_ztest

import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.4f}'.format

import warnings
warnings.filterwarnings("ignore")

# ANOVA. Однофакторный дисперсионный анализ.

$$SS_{total} = \sum_{i=1}^{n}\sum_{j=1}^{k}{(x_{ij} - M)^2}$$
$$SS_{bg} = n \cdot \sum_{j=1}^{k}{(M_j - M)^2}$$
$$SS_{wg} = \sum_{i=1}^{n}\sum_{j=1}^{k}{(x_{ij} - M_j)^2}$$
$$MS_{total} = \frac{SS_{total}}{n \cdot k -1}$$
$$MS_{bg} = \frac{SS_{bg}}{k-1}$$
$$MS_{wg} = \frac{SS_{wg}}{k(n-1)}$$
$$F = \frac{MS_{bg}}{MS_{wg}}$$

<p style="text-align: center;">$H_0:$ фактор не оказывает влияния (в среднем все группы не различаются)</p>
<p style="text-align: center;">$H_1:$ фактор оказывает влияния </p>

Условия: 
1. Нормальность распределения анализируемых групп или соответствие выборочных групп генеральным совокупностям с нормальным распределением.
2. Независимость (не связанность) распределения наблюдений в группах.
3. Наличие частоты (повторность) наблюдений.

# Задача 1

|Номер измерения|Ф_1|Ф_2|Ф_3|
|---|---|---|---|
|1|18|24|36|
|2|28|36|12|
|3|12|28|22|
|4|14|40|45|
|5|32|16|40|


In [48]:
df = pd.DataFrame({'Ф_1':[18, 28, 12, 14, 32], 
                   'Ф_2':[24, 36, 28, 40, 16], 
                   'Ф_3':[36, 12, 22, 45, 40]}, 
                  index=range(1, 6))

df

,Ф_1,Ф_2,Ф_3
1,18,24,36
2,28,36,12
3,12,28,22
4,14,40,45
5,32,16,40


#### Расчет вручную

In [49]:
k = df.shape[1]
n = df.shape[0]

M = df.sum().sum() / (n * k)

x_1_mean = df['Ф_1'].mean()
x_2_mean = df['Ф_2'].mean()
x_3_mean = df['Ф_3'].mean()

avg_list = [x_1_mean, x_2_mean, x_3_mean]

ss_total = ((df-M)**2).sum().sum()

ss_bg = n * sum([(x-M)**2 for x in avg_list])

ss_wg = ss_total - ss_bg

ms_total = ss_total/(n*k-1)
ms_bg = ss_bg/(k-1)
ms_wg = ss_wg/(n*k-k)

F = ms_bg/ms_wg

F

1.219525959367946

In [50]:
stats.f.sf(F, dfn=k-1, dfd=k*(n-1))

0.32949999192588353

$p-value$ выше $\alpha=0.05$. Следовательно, не можем отвергнуть нулевую гипотезу. **В среднем все группы не различаются**. 

#### рассчет с помощью пакета stats

In [51]:
stats.f_oneway(df['Ф_1'], df['Ф_2'], df['Ф_3'])

F_onewayResult(statistic=1.219525959367946, pvalue=0.32949999192588353)

# Задача 2

<table>
  <caption></caption> 
  <tr>
    <th rowspan="2" class="first">Гибрид (способ внесения)</th>
    <th colspan="5">Повторности</th>
  </tr>
  <tr>
    <td class="first">1</td>
    <td class="first">2</td>
    <td class="first">3</td>
    <td class="first">4</td>
    <td class="first">5</td>
  </tr>
  <tr>
    <td rowspan="1" class="first">Харпес(под культивацию)</td>
    <td>31.2</td>
    <td>28.6</td>
    <td>32.1</td>
    <td>32.1</td>
    <td>34.0</td>
  </tr>
    <td rowspan="1" class="first">Харпес(ленточный)</td>
    <td>28.1</td>
    <td>31.0</td>
    <td>32.3</td>
    <td>28.7</td>
    <td>39.6</td>
  <tr>
    <td rowspan="1" class="first">Харпес(под борование)</td>
    <td>28.6</td>
    <td>32.4</td>
    <td>26.9</td>
    <td>29.3</td>
    <td>32.1</td>
  </tr>
 </table>

In [53]:
df = pd.DataFrame([[31.2, 28.6, 32.1, 32.1, 34.0],
                   [28.1, 31.0, 32.3, 28.7, 39.6],
                   [28.6, 32.4, 26.9, 29.3, 32.1]], 
                  columns=range(1, 6), 
                  index=['Харпес(под культивацию)', 'Харпес(ленточный)', 'Харпес(под борование)'])

df

,1,2,3,4,5
Харпес(под культивацию),31.2000,28.6000,32.1000,32.1000,34.0000
Харпес(ленточный),28.1000,31.0000,32.3000,28.7000,39.6000
Харпес(под борование),28.6000,32.4000,26.9000,29.3000,32.1000


#### Расчет вручную

In [54]:
k = df.shape[1]
n = df.shape[0]

M = df.sum().sum() / (n * k)

x_1_mean = df[1].mean()
x_2_mean = df[2].mean()
x_3_mean = df[3].mean()
x_4_mean = df[4].mean()
x_5_mean = df[5].mean()

avg_list = [x_1_mean, x_2_mean, x_3_mean, x_4_mean, x_5_mean]

ss_total = ((df-M)**2).sum().sum()

ss_bg = n * sum([(x-M)**2 for x in avg_list])

ss_wg = ss_total - ss_bg

ms_total = ss_total/(n*k-1)
ms_bg = ss_bg/(k-1)
ms_wg = ss_wg/(n*k-k)

F = ms_bg/ms_wg

F

2.4126213592232886

In [55]:
stats.f.sf(F, dfn=k-1, dfd=k*(n-1))

0.11793662692972415

#### рассчет с помощью пакета stats

In [46]:
stats.f_oneway(df[1], df[2], df[3], df[4], df[5])

F_onewayResult(statistic=2.412621359223302, pvalue=0.11793662692972288)